In [29]:
import pandas as pd
import numpy as np

In [30]:
import warnings
warnings.filterwarnings('ignore')

In [31]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

import sys
import os

path= "C:\\Users\\Aman\\Desktop\\kifyaw8-9\\src"
sys.path.append(os.path.abspath(path=path))

try:
    from data_preprocessing import DataPreprocessing
except:
    print("Import failure")

In [32]:
df_ip = pd.read_csv('/content/drive/MyDrive/week8-9/IpAddress_to_Country.csv')
class DataPreprocessing:
    def __init__(self):
        pass
    def to_date(self,column_list,data):
        self.data = data
        for column_name in column_list:
            self.data[column_name] = pd.to_datetime(self.data[column_name])
        return self.data
    def assign_country_code(self,df_Fraud):
        country_list = []  # Store results

        for fraud_ip in df_Fraud['ip_address']:
            # Filter the correct country based on IP range
            match = df_ip.loc[(df_ip['lower_bound_ip_address'] <= fraud_ip) &
                            (df_ip['upper_bound_ip_address'] >= fraud_ip), 'country']

            # Append the first matched country (if any), otherwise append None
            country_list.append(match.iloc[0] if not match.empty else None)

        df_Fraud['country'] = country_list  # Add new column to df_Fraud
        return df_Fraud


In [33]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier


In [47]:
from sklearn.metrics import accuracy_score

In [34]:

data_processing = DataPreprocessing()




df_fraud = pd.read_csv('/content/drive/MyDrive/week8-9/Fraud_Data.csv')
# df_country_code = pd.read_csv('C:\\Users\\Aman\\Desktop\\kifyaw8-9\\data\\raw\\IpAddress_to_Country.csv')


In [35]:
def feature_enginerring(df_fraud):
    df = data_processing.assign_country_code(df_fraud)
    # Convert timestamps to datetime
    df['signup_time'] = pd.to_datetime(df['signup_time'])
    df['purchase_time'] = pd.to_datetime(df['purchase_time'])

    # Calculate transaction frequency per user
    transaction_frequency = df.groupby('device_id').size().reset_index(name='transaction_frequency')

    # Calculate transaction velocity (time between transactions)
    df = df.sort_values(by=['device_id', 'purchase_time'])  # Sort by user and purchase time
    df['time_diff'] = df.groupby('device_id')['purchase_time'].diff().dt.total_seconds()  # Time difference in seconds

    # Calculate average transaction velocity per user (transactions per second)
    transaction_velocity = df.groupby('device_id')['time_diff'].mean().reset_index(name='avg_transaction_velocity')

    # Merge frequency and velocity into one DataFrame
    user_transaction_stats = pd.merge(transaction_frequency, transaction_velocity, on='device_id', how='left')
    user_transaction_stats['avg_transaction_velocity'].fillna(0,inplace=True)
    df['time_diff'].fillna(0, inplace=True)

    # Merge user_transaction_stats with the original dataset
    df_stats = pd.merge(df, user_transaction_stats, on='device_id', how='left')

    # Substituting nan values in country columns with unidentified
    df['country'].fillna('Unidentified', inplace=True)

    # Extract hour of the day and day of the week
    df_stats['hour_of_day'] = df_stats['purchase_time'].dt.hour
    df_stats['day_of_week'] = df_stats['purchase_time'].dt.dayofweek  # Monday=0, Sunday=6

    # Extract useful features
    df_stats['signup_hour'] = df_stats['signup_time'].dt.hour
    df_stats['signup_day'] = df_stats['signup_time'].dt.dayofweek

    # encodeing device id with a hash encodeing
    df_stats['device_id'] = df_stats['device_id'].apply(lambda x: hash(x) % 10)
    df_stats['country'] = df_stats['country'].apply(lambda x: hash(x) % 10)
    df_stats['sex'] = df_stats['sex'].map({'M': 1, 'F':0})

    return df_stats.drop(['purchase_time','signup_time'], axis=1)


In [36]:
preprocesser = ColumnTransformer([
    ('scaler', StandardScaler(),['purchase_value', 'age', 'time_diff', 'transaction_frequency',
                    'avg_transaction_velocity', 'hour_of_day', 'day_of_week', 'country']),
    ('onehot', OneHotEncoder(handle_unknown='infrequent_if_exist', sparse_output=False), ['browser','source'])

])

randomforest = Pipeline([
    ('feature_adder', FunctionTransformer(feature_enginerring, validate=False)),
    ('preprocessor', preprocesser,),
    ('model', RandomForestClassifier(n_estimators=30, max_depth=2))
])

gradientboosting = Pipeline([
    ('feature_adder', FunctionTransformer(feature_enginerring, validate=False)),
    ('preprocessor', preprocesser,),
    ('model', GradientBoostingClassifier(n_estimators=10, n_iter_no_change=10))
])

logisticreg = Pipeline([
    ('feature_adder', FunctionTransformer(feature_enginerring, validate=False)),
    ('preprocessor', preprocesser,),
    ('model', LogisticRegression(random_state=42))
])
decisiontree = Pipeline([
    ('feature_adder', FunctionTransformer(feature_enginerring, validate=False)),
    ('preprocessor', preprocesser,),
    ('model', DecisionTreeClassifier(random_state=42))
])

In [37]:
x = df_fraud.drop('class', axis=1)
y = df_fraud['class']

x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.2)


In [42]:

logisticreg.fit(x_train,y_train)



Pipeline(steps=[('feature_adder',
                 FunctionTransformer(func=<function feature_enginerring at 0x7c805b067060>)),
                ('preprocessor',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  ['purchase_value', 'age',
                                                   'time_diff',
                                                   'transaction_frequency',
                                                   'avg_transaction_velocity',
                                                   'hour_of_day', 'day_of_week',
                                                   'country']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                sparse_output=False),
                                                  ['browser', 'source'])])),
                ('model', LogisticRegression(random_state=42))])

In [44]:
decisiontree.fit(x_train, y_train)


Pipeline(steps=[('feature_adder',
                 FunctionTransformer(func=<function feature_enginerring at 0x7c805b067060>)),
                ('preprocessor',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  ['purchase_value', 'age',
                                                   'time_diff',
                                                   'transaction_frequency',
                                                   'avg_transaction_velocity',
                                                   'hour_of_day', 'day_of_week',
                                                   'country']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                sparse_output=False),
                                                  ['browser', 'source'])])),
                ('model', DecisionTreeClassifier(random_state=42))])

In [ ]:

pipeline.fit(x_train,y_train)

Pipeline(steps=[('feature_adder',
                 FunctionTransformer(func=<function feature_enginerring at 0x000001604B4AA7A0>)),
                ('preprocessor',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  ['purchase_value', 'age',
                                                   'time_diff',
                                                   'transaction_frequency',
                                                   'avg_transaction_velocity',
                                                   'hour_of_day', 'day_of_week',
                                                   'country']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                sparse_output=False),
                                                  ['browser', 'source'])])),
                ('model',
                 RandomForestClassifier(max_depth=2, n_estimators=30))])

In [ ]:
pipeline2.fit(x_train, y_train)


Pipeline(steps=[('feature_adder',
                 FunctionTransformer(func=<function feature_enginerring at 0x000001604B4AA7A0>)),
                ('preprocessor',
                 ColumnTransformer(transformers=[('scaler', StandardScaler(),
                                                  ['purchase_value', 'age',
                                                   'time_diff',
                                                   'transaction_frequency',
                                                   'avg_transaction_velocity',
                                                   'hour_of_day', 'day_of_week',
                                                   'country']),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='infrequent_if_exist',
                                                                sparse_output=False),
                                                  ['browser', 'source'])])),
                ('model',
                 GradientBoostingClassifier(n_estimators=10,
                                            n_iter_no_change=10))])

In [ ]:
y_pred = pipeline.predict(x_test)
from sklearn.metrics import accuracy_score
accu_score = accuracy_score(y_test, y_pred)
accu_score

0.9088442576845449

In [ ]:
y_pred_gbc =  pipeline2.predict(x_test)
gbc_score = accuracy_score(y_test, y_pred_gbc)
gbc_score

0.9088442576845449

In [48]:
y_pred_gbc =  logisticreg.predict(x_test)
gbc_score = accuracy_score(y_test, y_pred_gbc)
gbc_score

0.906859014657711

In [49]:
y_pred_gbc =  decisiontree.predict(x_test)
gbc_score = accuracy_score(y_test, y_pred_gbc)
gbc_score

0.8086556595969957

In [50]:
import joblib
import pickle
import json

joblib.dump(logisticreg,'logisticreg.pkl')
joblib.dump(decisiontree,'decisiontree.pkl')

['decisiontree.pkl']